In [288]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
import pickle
from sklearn import preprocessing
from scipy import signal
from sklearn.preprocessing import OneHotEncoder
import re

#filename queue
filename_queue=glob.glob('./rawdata/data[1-5].csv')

#Create return path
return_path='./rawdata/data.csv'
create_file=open(return_path,'w')

In [1]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
#index=[1,2,3,11,14]
index=[1,2,3]
label_count=np.zeros(17)
traininglabel_count=np.zeros(17)
testlabel_count=np.zeros(17)
n_data=[]

#read file from filename queue
for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        labl=int(row[3])

        label_count[labl]+=4
        if labl not in index:
            for x in range(4):
                
                #Extract sensor data and smoothing
                X_acc=list(row[4*i+4+x] for i in range(150))
                X_acc=signal.savgol_filter(X_acc,5,3)
                
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Y_acc=signal.savgol_filter(Y_acc,5,3)
                
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                Z_acc=signal.savgol_filter(Z_acc,5,3)
                
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                X_gyro=signal.savgol_filter(X_gyro,5,3)
                
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Y_gyro=signal.savgol_filter(Y_gyro,5,3)
                
                Z_gyro=list(row[4*i+3004+x] for i in range(150))
                Z_gyro=signal.savgol_filter(Z_gyro,5,3)
                
                #construct window
                window=np.array([X_acc[0:25],Y_acc[0:25],Z_acc[0:25],X_gyro[0:25],Y_gyro[0:25],Z_gyro[0:25],
                                  X_acc[25:50],Y_acc[25:50],Z_acc[25:50],X_gyro[25:50],Y_gyro[25:50],Z_gyro[25:50],
                                  X_acc[50:75],Y_acc[50:75],Z_acc[50:75],X_gyro[50:75],Y_gyro[50:75],Z_gyro[50:75],
                                  X_acc[75:100],Y_acc[75:100],Z_acc[75:100],X_gyro[75:100],Y_gyro[75:100],Z_gyro[75:100],
                                  X_acc[100:125],Y_acc[100:125],Z_acc[100:125],X_gyro[100:125],Y_gyro[100:125],Z_gyro[100:125],
                                  X_acc[125:150],Y_acc[125:150],Z_acc[125:150],X_gyro[125:150],Y_gyro[125:150],Z_gyro[125:150]])
                
                window2=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
                
                #5 features  what else?2
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)
                
                #Drop outliers1 : want to use tnse further
                if(labl!=-1):
                    xm=window_stddev[[0,6,12,18,24,30]].mean(axis=0)
                    ym=window_stddev[[1,7,13,19,25,31]].mean(axis=0)
                    zm=window_stddev[[2,8,14,20,26,32]].mean(axis=0)
                    #print('total',np.mean([xm,ym,zm]),'labl:', labl)
                    action=np.mean([xm,ym,zm])
                if(action<0.65):
                    label_count[labl]-=1
                    break
                

                #construct window feature
                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)
                
                #construct training and test set
                if (label_count[labl]%5!=1):
                    training_set.append(window_feature)
                    training_label.append(labl)
                    traininglabel_count[labl]+=1

                else:
                    test_set.append(window_feature)
                    test_label.append(labl)
                    testlabel_count[labl]+=1
        

NameError: name 'np' is not defined

In [290]:
# load svm
filename='./svm.sav'
svm = pickle.load(open(filename, 'rb'))

# load knn
filename='./knn.sav'
knn = pickle.load(open(filename, 'rb'))

# load rf
filename='./rf.sav'
rf = pickle.load(open(filename, 'rb'))

# load ETC
filename='./ETC.sav'
ETC = pickle.load(open(filename, 'rb'))

In [291]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
X=training_set
y=training_label
RF = RandomForestClassifier(200)
RF.fit(X, y)
test_pred_proba=RF.predict_proba(test_set)

#print((test_pred_proba))
#for row in test_pred_proba:
#    print(np.max([row]))
test_pred3=RF.predict(test_set)
print(accuracy_score(test_pred3,test_label))
print(confusion_matrix(test_pred3, test_label))



ohc=OneHotEncoder(categories='auto')
onehot_pred3=test_pred3.reshape(-1,1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
print(onehot_pred3)

0.8726415094339622
[[21  0  0  0  0  0  0  0  0]
 [ 0 21  4  0  0  0  0  0  0]
 [ 3  3 40  0  4  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0]
 [ 0  0  0  0  8  0  0  0  0]
 [ 0  0  0  0  0  8  0  0  0]
 [ 0  0  0  0  0  0 20  0  0]
 [ 0  0  0  1  0  4  8 48  0]
 [ 0  0  0  0  0  0  0  0  8]]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [286]:
#SVM
#prepare the data
X=preprocessing.minmax_scale(training_set)
y=training_label
#prepare the model
SVC = sklearn.svm.SVC(gamma='scale',tol=0.1,probability=True)

SVC.fit(X,y)


test_pred1=SVC.predict(preprocessing.minmax_scale(test_set))
test_pred_proba=SVC.predict_proba(preprocessing.minmax_scale(test_set))

ohc=OneHotEncoder(categories='auto')
onehot_pred1=test_pred1.reshape(-1,1)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
print(test_pred1)
print(onehot_pred1)
#print(confusion_matrix(test_pred1, test_label))

print(accuracy_score(test_pred1,test_label))
print(SVC.n_support_)
#confusion_matrix(test_pred1, test_label)

[12 12 12 12  9  9  9  9 12 12 12 12 10 10 10 10  9  9  9  9 12 12 12 12
  7  7  7  7  6  6  6  6  6  6  6  6  0  0  0  0  5  5  5  5  0  0  0  0
 12 12 12 12 12 12 12 12  0  0  0  0 12 12 12 12  6  6  6  6 12 12 12 12
 12 12 12 12  7  7  7  7  0  0  0  0 12 12 12 12 12 12 12 12  6  6  6  6
 12 12 12 12 12 12 12 12 12 12  0  0  5  5  5  5  0  0  0  0  6  6  6  6
  0  0  0  0  6  6  6  6 10 10 10 10 13 13 13 13  8  8  8  8  8  8  8  8
  0  0  0  0 12 12 12 12 12 12 12 12  5  5  5  5 12 12 12 12  6  6  6  6
  5  5  5  5 12 12 12 12  6  6  6  6  6  6  6  6 10 10 10 10 10 10 10 10
  5  5  5  5 10 10 10 10  0  0  0  0  0  0  0  0  6  6  6  6]
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
0.8301886792452831
[ 63   4  72  88  43  34  45  76 132  18   8]


In [268]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
#window=preprocessing.quantile_transform(window)

X=training_set
X=preprocessing.quantile_transform(training_set)
test_set2=preprocessing.quantile_transform(test_set)
y=training_label
KNN = NearestCentroid()
KNN.fit(X, y)
test_pred2=KNN.predict(test_set2)
print(accuracy_score(test_pred2,test_label))

#onehot_pred2=test_pred2.reshape(-1,1)
#onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)


0.6404494382022472


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)
/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (356). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [279]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
X=training_set
y=training_label
RF = RandomForestClassifier(200)
RF.fit(X, y)
test_pred_proba=RF.predict_proba(test_set)
print(np.shape(test_pred_proba))
#print(np.argmax(x) for x in test_pred_proba)
test_pred3=RF.predict(test_set)
#for x in test_pred_proba:
#    print(x)
print(accuracy_score(test_pred3,test_label))
print(confusion_matrix(test_pred3, test_label))
#onehot_pred3=test_pred2.reshape(-1,1)
#onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
#print(onehot_pred3)

(356, 13)
0.7808988764044944
[[36  0  0  4  0  0  1  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3 24  8  0  2  0  0  0  0  0  0  0]
 [ 3  0  5 36  0  4  0  0  0  0  0  0  0]
 [ 1  0  0  0 20  0  0  0  0  4  0  0  4]
 [ 0  1  3  4  0 18  0  4  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  4  0  0  0]
 [ 0  0  0  0  0  0  0 48  0  4  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0]
 [ 4  0  0  0  4  0  7  4  0 44  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  8  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  4  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 12]]


In [270]:
array=np.array(test_pred_proba)
newpred2=np.zeros(len(test_pred3))
for x in range(len(test_pred_proba)):
    if(np.max(test_pred_proba[x])>0.3):
        newpred2[x]=test_pred3[x]
    else:
        newpred2[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred2))

(356,)


In [271]:
print(accuracy_score(newpred2,test_label))

0.7331460674157303


In [272]:
#ensemble 1 : unanimous consensus
#ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
#ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
#print(ensemble1_pred)

In [273]:
#ensemble 2 : majority consensus
#ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
#ensemble2_pred= ensemble2_pred.astype('int64') 
#ensemble2_pred= ensemble2_pred.astype('float64') 
#ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
#print(ensemble2_pred)
#print(test_label)
#print(accuracy_score(ensemble2_pred,test_label))


In [274]:
#save the model

from joblib import dump, load
filename='./svm.sav'
pickle.dump(SVC,open(filename,'wb'))

filename='./knn.sav'
pickle.dump(KNN,open(filename,'wb'))
dump(KNN,'knn.joblib')

filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

filename='./ETC.sav'
pickle.dump(ETC,open(filename,'wb'))
dump(ETC,'ETC.joblib')


['ETC.joblib']

In [275]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
X=training_set
y=training_label

ETC=ExtraTreesClassifier(150)
DTC=DecisionTreeClassifier()

ETC.fit(X,y)
DTC.fit(X,y)
test_pred4=ETC.predict(test_set)
test_pred5=DTC.predict(test_set)
print(accuracy_score(test_pred4,test_label))



0.7865168539325843


In [276]:
test_pred_proba2=ETC.predict_proba(test_set)
array=np.array(test_pred_proba2)
newpred3=np.zeros(len(test_pred4))
for x in range(len(test_pred_proba2)):
    if(np.max(test_pred_proba2[x])>0.2):
        newpred3[x]=test_pred4[x]
    else:
        newpred3[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred3))
print(newpred3)
print(accuracy_score(newpred3,test_label))

(356,)
[12. 12. 12. 12.  9.  9.  9.  9.  7.  7.  7.  7. 10. 10. 10. 10. 12. 12.
 12. 12. 10. 10. 10. 10.  7.  7.  7.  7.  6.  6.  6.  6.  6.  6.  6.  6.
  0.  0.  0.  0.  5.  5.  5.  5.  6.  6.  6.  6. 10. 10. 10. 10. 12. 12.
 12. 12.  0.  0.  0.  0. 10. 10. 12. 10.  6.  6.  6.  6. 12. 12. 12. 12.
 10. 10. 10. 10.  7.  7.  7.  7.  0.  0.  0.  0. 12. 12. 12. 12. 12. 12.
 12. 12.  6.  6.  6.  6. 12. 12. 12. 12. 12. 12. 12. 12.  7.  7.  6.  7.
  5.  5.  5.  5.  0.  0.  0.  0.  6.  6.  6.  6.  0.  0.  0.  0.  5.  5.
  5.  5. 10. 10. 10. 10. 13. 13. 13. 13.  8.  8.  8.  8.  8.  8.  8.  8.
 13. 13. 13. 13. 12. 12. 12. 12. 12. 12. 12. 12.  5.  5.  5.  5. 12. 12.
 12. 12.  5.  5.  5.  5.  5.  5.  5.  5. 12. 12. 12. 12.  5.  5.  5.  5.
  6.  6.  6.  6.  8.  6.  6.  8.  8.  8.  8.  8. 12. 12. 12. 12. 10. 10.
 -1. -1.  5.  5.  5.  5.  0. 12. 12.  0.  5.  5.  5.  5.  7.  7.  7.  7.
  8.  8.  8.  8.  0.  0.  0.  0. -1. -1. -1. -1.  0.  0.  0.  0.  6.  6.
  6.  6.  0.  0.  0.  0.  7.  7.  7.  7.  0.

In [277]:
accuracy_per_label=[]

accuracy_per_label=confusion_matrix(test_pred4, test_label)/testlabel_count
accuracy_per_label.diagonal()
accuracy_per_label=confusion_matrix(test_pred5, test_label)/testlabel_count
accuracy_per_label.diagonal()

ValueError: operands could not be broadcast together with shapes (13,13) (17,) 

In [ ]:
index=[2,11,8]
